# C'est la deuxième partie du projet : construire un classifieur (CIMDB) qui permet de deviner le IMDB rating du film à partir d’un dialogue dans le film.


# **1)Importez les bibliothèques:**

In [ ]:
import re
import nltk
import string

import pandas as pd
import numpy as np

from numpy import asarray
from numpy import zeros

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import f1_score



from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer, one_hot

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding , Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,Bidirectional,SpatialDropout1D,GlobalMaxPooling1D
from tensorflow.python.keras import Sequential


from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


from google.colab import drive 
drive.mount('/content/gdrive')


Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# **2) Load Data:**

Le fichier movie_titles_metadata.txt contient des informations sur chaque film , on a besoin du 'IMDB rating' le rate de chaque film

In [ ]:
f1 = open('/content/gdrive/My Drive/movie_titles_metadata.txt','rb')

L=[]
for l in f1 :
  L.append(l)
  
Metadata=[]
for l in L:
    ch=str(l)
    gn=[]
    A=[]
    a=ch.split(" +++$+++ ")
    
    a[3]=round(float(a[3]))
    A.append(a[3])

    
    Metadata.append(A[0])


In [ ]:
len(Metadata)

617

In [ ]:
Metadata[0]

7

Pour une représentation matricielle binaire du rate (catégorique):


In [ ]:
labels = to_categorical(Metadata)

In [ ]:
labels[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)

Ce deuxième fichier contient les texts des charactères

In [ ]:

f2 = open('/content/gdrive/My Drive/movie_lines.txt','rb')

L2=[]
for l in f2 :
  L2.append(l)

line=[]
for l in L2:
    ch=str(l)
    gn=[]
    a=ch.split(" +++$+++ ")
    if len(a)!=5:
      continue
    else:

        a[0]=a[0][2:] 
        a[4]=a[4][:-3]  
    
    line.append(a)

In [ ]:
movie_lines=pd.DataFrame(line,columns = [ 'lineId','charId','movieId','char_Name','line'])
movie_lines.head(10)

,lineId,charId,movieId,char_Name,line
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.
5,L924,u2,m0,CAMERON,Wow
6,L872,u0,m0,BIANCA,Okay -- you're gonna need to learn how to lie.
7,L871,u2,m0,CAMERON,No
8,L870,u0,m0,BIANCA,I\'m kidding. You know how sometimes you just...
9,L869,u0,m0,BIANCA,Like my fear of wearing pastels?


# **3)Clean Data**

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text
# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)



In [ ]:
movie_lines['line']=movie_lines['line'].apply(lambda x:clean_text(x))
movie_lines['line']=movie_lines['line'].apply(lambda x:remove_stopwords(x))


In [ ]:
movie_lines.head(10)

,lineId,charId,movieId,char_Name,line
0,L1045,u0,m0,BIANCA,
1,L1044,u2,m0,CAMERON,
2,L985,u0,m0,BIANCA,hope
3,L984,u2,m0,CAMERON,okay
4,L925,u0,m0,BIANCA,let go
5,L924,u2,m0,CAMERON,wow
6,L872,u0,m0,BIANCA,okay gonna need learn lie
7,L871,u2,m0,CAMERON,
8,L870,u0,m0,BIANCA,kidding know sometimes become persona know quit
9,L869,u0,m0,BIANCA,like fear wearing pastels


In [ ]:
list(movie_lines[movie_lines.movieId=='m602'].line)

['oh wish woman would screwed woman loved think way',
 'mean officially woman',
 'well',
 'would stop',
 'said hate darcy oh boy men stupider true',
 'darcy since']

In [ ]:
movie_diag=[]
i=''
word_nbr=[]
for movie in movie_lines.movieId:

  if movie==i:
    continue
  diag=''
  for d in (movie_lines[movie_lines.movieId==movie].line):
    diag=diag+d
  movie_diag.append(diag)
  word_nbr.append(len(diag))
  i=movie

In [ ]:
df=pd.DataFrame(data=movie_diag,columns=['lines'])


In [ ]:
df.head()

,lines
0,hopeokaylet gowowokay gonna need learn liekidd...
1,far say also like smell sea around smells like...
2,officers killer duty arrestkill someone famous...
3,trying get hopecontinuedsorry dr smyslov reall...
4,great greatput air tires even sell gas needcha...


In [ ]:
df.shape

(617, 1)

In [ ]:
df['nbr']=[len(l.split())for l in df.lines]

In [ ]:
df.head()

,lines,nbr
0,hopeokaylet gowowokay gonna need learn liekidd...,1857
1,far say also like smell sea around smells like...,891
2,officers killer duty arrestkill someone famous...,2460
3,trying get hopecontinuedsorry dr smyslov reall...,1143
4,great greatput air tires even sell gas needcha...,2519


In [ ]:
movie_diag[66]

'would find kathywowuh see signature hang note see addendum checkingsigned sponsoring officergotwickwire thomas dane see findokayrank favor pull transfer orderlieutensay nameadministration ensign blondellwould guess hold coffee klatches three us get together one time guys think kind uprisingmean manyfriend friends knowwoman sawaccused wishknow single reason father navy old time recruiting poster den showed girl trying sailor uniform saying gee wish man would join navy maybe 10 years old first saw even felt wrong made mad think month gone thought poster gee wish manwomenask somethin lieutenant come mean kinda curiousheartake away san clemente island half guys quit come back supposed hell halfproblem lieutenantthank ensignb q south side take starboard tack doorset good plates lunch office follow details jumpingcoursesecurity hell talkin base secureuh v p security arrangements generally take time senatorjordan dear treatingstanding right officeawright girl doin anywayprerogative senatorwe

# **3) GLOVE :**

L'étape de préparation des données:

Keras fournit une classe Tokenizer qui peut être adaptée aux données d'apprentissage, peut convertir le texte en séquences de manière cohérente en appelant la méthode textes_to_sequences () sur la classe Tokenizer.

In [ ]:
max_features =np.max(df.nbr) # how many unique words to use (i.e num rows in embedding vector)
X=df['lines']
# prepare tokenizer
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X))
# integer encode lines
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X)


Ensuite, on importe le fichier GloVe word embedding au tant que dictionnaire .


In [ ]:
embeddings_index = dict()
f = open("/content/gdrive/My Drive/glove.6B.100d.txt")
#f = open('glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()


Ensuite, On crée une matrice d'une embedding pour chaque mot du dataset. On peut le faire en énumérant tous les mots uniques dans le Tokenizer.word_index et en localisant le vecteur de poids d'embedding à partir de GloVe embedding.


In [ ]:
vocabulary_size=16078

embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

On peut maintenant construire les modèles .

In [ ]:
train_X, test_X, train_y , test_y = train_test_split(X,labels ,test_size=0.2, random_state=20)


# **4) Models**

 **LSTM**

In [ ]:
model_lstm = Sequential()
model_lstm.add(Embedding(vocabulary_size,100,weights = [embedding_matrix],input_length=train_X.shape[1],trainable = False))
model_lstm.add(SpatialDropout1D(0.2))
model_lstm.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_lstm.add(Dense(52,activation = 'relu'))
model_lstm.add(Dense(10, activation='softmax'))
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 100

history = model_lstm.fit(train_X, train_y, epochs=epochs, batch_size=batch_size,validation_split=0.1)


Epoch 1/5
5/5 [==============================] - 80s 16s/step - loss: 2.2165 - accuracy: 0.1806 - val_loss: 2.0098 - val_accuracy: 0.3200
Epoch 2/5
5/5 [==============================] - 79s 16s/step - loss: 1.8856 - accuracy: 0.2889 - val_loss: 1.7625 - val_accuracy: 0.2400
Epoch 3/5
5/5 [==============================] - 79s 16s/step - loss: 1.6855 - accuracy: 0.3318 - val_loss: 1.7054 - val_accuracy: 0.2800
Epoch 4/5
5/5 [==============================] - 79s 16s/step - loss: 1.6074 - accuracy: 0.3251 - val_loss: 1.6777 - val_accuracy: 0.2400
Epoch 5/5
5/5 [==============================] - 79s 16s/step - loss: 1.6052 - accuracy: 0.3589 - val_loss: 1.6724 - val_accuracy: 0.2000


In [ ]:
pred_lstm= model_lstm.predict(test_X , verbose=1)



4/4 [==============================] - 5s 1s/step


In [ ]:
pred_lstm[0]

array([0.00647645, 0.00271519, 0.00332619, 0.00126094, 0.0344288 ,
       0.07641193, 0.25730637, 0.2819725 , 0.3216231 , 0.01447863],
      dtype=float32)

Cet prédiction est faite sur la base d'une valeur seuil de 0,5, ce qui signifie que les probabilités supérieures ou égales à 0,5 ont été converties en 1 et les autres en 0.
On va modifier cette valeur seuil à 0.2 pour améliorer le score du modèle:




In [ ]:
# predict probabilities
y_pred_prob = model_lstm.predict_proba(test_X)

Instructions for updating:
Please use `model.predict()` instead.


In [ ]:
t = 0.2 # threshold value
y_pred_new = (y_pred_prob >= t).astype(int)

In [ ]:
f1_score(test_y, y_pred_new, average="micro")


0.4210526315789474

 **CNN**

In [ ]:
model_cnn = Sequential()
model_cnn.add(Embedding(vocabulary_size,100,weights = [embedding_matrix],input_length=train_X.shape[1],trainable = False))
model_cnn.add(Conv1D(100, 5, activation='relu'))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(55, activation='relu'))
model_cnn.add(Dense(10, activation='softmax'))

model_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 90

history = model_cnn.fit(train_X, train_y, epochs=epochs, batch_size=batch_size,validation_split=0.1)


Epoch 1/5
5/5 [==============================] - 14s 3s/step - loss: 2.0113 - accuracy: 0.2144 - val_loss: 1.7382 - val_accuracy: 0.3400
Epoch 2/5
5/5 [==============================] - 14s 3s/step - loss: 1.6098 - accuracy: 0.3702 - val_loss: 1.6503 - val_accuracy: 0.2800
Epoch 3/5
5/5 [==============================] - 14s 3s/step - loss: 1.4776 - accuracy: 0.4989 - val_loss: 1.6306 - val_accuracy: 0.4000
Epoch 4/5
5/5 [==============================] - 14s 3s/step - loss: 1.3954 - accuracy: 0.5734 - val_loss: 1.6193 - val_accuracy: 0.2800
Epoch 5/5
5/5 [==============================] - 14s 3s/step - loss: 1.3121 - accuracy: 0.6817 - val_loss: 1.6027 - val_accuracy: 0.3400


In [ ]:
pred= model_cnn.predict(test_X , verbose=1)
# predict probabilities
y_pred_prob = model_cnn.predict_proba(test_X)
t = 0.2 # threshold value
y_pred_new = (y_pred_prob >= t).astype(int)

4/4 [==============================] - 1s 303ms/step


In [ ]:
f1_score(test_y, y_pred_new, average="micro")


0.4201312910284464

**SVM**

In [ ]:
emb = Sequential()
emb.add(Embedding(vocabulary_size,100,weights = [embedding_matrix],input_length=train_X.shape[1],trainable = False))
emb.add(GlobalMaxPooling1D())

In [ ]:
Train_X_emb=emb.predict(train_X)
Test_X_emb=emb.predict(test_X)

In [ ]:
SVM = OneVsRestClassifier(SVC(probability=True))
SVM.fit(Train_X_emb,train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))


OneVsRestClassifier(estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                  class_weight=None, coef0=0.0,
                                  decision_function_shape='ovr', degree=3,
                                  gamma='scale', kernel='rbf', max_iter=-1,
                                  probability=True, random_state=None,
                                  shrinking=True, tol=0.001, verbose=False),
                    n_jobs=None)

In [ ]:
predictions_SVM = SVM.predict(Test_X_emb)

In [ ]:
y_pred_prob = SVM.predict_proba(Test_X_emb)
t = 0.2 # threshold value
y_pred_new = (y_pred_prob >= t).astype(int)

f1_score(test_y, y_pred_new, average="micro")

0.4059196617336152